In [1]:
import numpy as np
import pandas as pd  
import time

In [2]:
dfUser = pd.read_csv('../data_csv/User.csv')
dfOrderDetail = pd.read_csv('../data_csv/OrderDetail.csv')
dfOrderDetail = dfOrderDetail.rename(columns= {"code" : 'code_order' , 'name' : 'name_order' , 'started_at' : 'started_at_order' , 'completed_at' : 'completed_at_order'})
dfCustomerDetail = pd.read_csv('../data_csv/CustomerDetail.csv')
dfCustomerDetail = dfCustomerDetail.drop(columns= {'id' , 'creator_id' , 'created_at' , 'updated_at' , 'deleted_at' , 'email' , 'phone' , 'fax' , 'tax_number' , 'address' , 'media_id' , 'user_id'})
dfCustomerDetail = dfCustomerDetail.rename(columns= {'code' : 'code_cus' , 'name' : 'name_cus' })
# dfCustomerDetail.head(2)
dfOrderDetail = dfOrderDetail.merge(dfCustomerDetail , on = 'customer_id' , how = 'left')
# dfOrderDetail[dfOrderDetail['code_order'] == 'DYC-US-230210-E11even-705-7009']['started_at_order']


In [3]:
dfProductionTracking = pd.read_csv('../data_csv/ProductionTracking.csv')
dfProductionTrackingDetail = pd.read_csv('../data_csv/ProductionTrackingDetail.csv')
dfManufactureDeptDetail = pd.read_csv('../data_csv/ManufactureDeptDetail.csv')
dfProductionNotExistOrder = pd.read_csv('../data_csv/ProductionNotExistOrder.csv')

dfManufactureDeptDetail = dfManufactureDeptDetail.rename(columns= {'code' : 'code_dept' , 'name' : 'name_dept'})
dfProductionNotExistOrder = dfProductionNotExistOrder.rename(columns= {'id' : 'prod_not_exist_order_id','created_at' : 'not_exist_create' , 'name' : 'name_not_exist'}).drop(columns= {'updated_at' , 'deleted_at' , 'qrcode'})

In [4]:
dfProductionTracking = dfProductionTracking.merge(dfOrderDetail[['code_order' , 'name_order' , 'order_id' , 'name_cus' , 'quantity' , 'started_at_order' , 'completed_at_order']], on = 'order_id' , how = 'left')
dfProductionTracking = dfProductionTracking.merge(dfManufactureDeptDetail[['code_dept' , 'name_dept' , 'manufacture_dept_id']], on = 'manufacture_dept_id' , how = 'left')
dfProductionTracking = dfProductionTracking.merge(dfProductionNotExistOrder, on = 'prod_not_exist_order_id' , how = 'left')

In [5]:
dfProductionTracking.head(2)

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,...,name_cus,quantity,started_at_order,completed_at_order,code_dept,name_dept,name_not_exist,not_exist_create,buyer_name,style_name
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
1,2,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,2.0,8,301.0,0.0,0.0,...,NaN,NaN,NaN,NaN,LEAN 4,May 4,PO38046SAND,2022-10-22 03:00:00+00:00,SA,PO 38046 Charter Hat Sand


In [6]:
dfProductionTrackingFix = dfProductionTracking.drop(columns= {'id','updated_at', 'deleted_at' , 'order_id' , 'manufacture_dept_id', 'order_id', 'prod_not_exist_order_id' })
dfProductionTrackingFix

,created_at,target_quantity,actual_quantity,complete_percent,code_order,name_order,name_cus,quantity,started_at_order,completed_at_order,code_dept,name_dept,name_not_exist,not_exist_create,buyer_name,style_name
0,2022-10-22 03:00:00+00:00,927.0,318.0,34.30,NaN,NaN,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
1,2022-10-22 03:00:00+00:00,301.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,LEAN 4,May 4,PO38046SAND,2022-10-22 03:00:00+00:00,SA,PO 38046 Charter Hat Sand
2,2022-10-22 03:00:00+00:00,268.0,2.0,0.75,NaN,NaN,NaN,NaN,NaN,NaN,LEAN 5,May 5,PO38050ROL_TAN,2022-10-22 03:00:00+00:00,SA,PO38050 Kids Clear Creek Boonie
3,2022-10-22 03:00:00+00:00,289.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,LEAN 6,May 6,PO38055_ULTR_PUMI,2022-10-22 03:00:00+00:00,SA,PO38056_Ultra adventure Pumice
4,2022-10-23 03:00:00+00:00,1458.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729,2023-03-03 07:51:08+00:00,994.0,140.0,14.08,TTD-221112-SP-V-S23SUC-BH11-LK,TTD-221112-SP-V-S23SUC-BH11-LK,"CapBoy Trading Co.,Ltd",4000.0,2023-02-01,2023-03-17,LEAN 4,May 4,NaN,NaN,NaN,NaN
1730,2023-03-03 08:00:02+00:00,336.0,19.0,5.65,DYC-CH1-221222-Lavsanne Palace,DYC-CH1-221222-Lavsanne Palace,DYC Sàrl,102.0,2023-02-08,2023-03-06,LEAN 1,May 1,NaN,NaN,NaN,NaN
1731,2023-03-03 08:51:01+00:00,163.0,88.0,53.99,DYC-US-230126-Blacktiph-690-Opt07,DYC-US-230126-Blacktiph-690-Opt07,DESIGN YOUR CAPS USA LLC,250.0,2023-02-01,2023-03-07,LEAN 1,May 1,NaN,NaN,NaN,NaN
1732,2023-03-03 09:21:02+00:00,83.0,100.0,120.48,DYC-US-221222-Takuma-680,DYC-US-221222-Takuma-680,DESIGN YOUR CAPS USA LLC,100.0,2023-02-15,2023-03-06,LEAN 1,May 1,NaN,NaN,NaN,NaN


In [7]:
dfProductionTrackingFix = dfProductionTracking.drop(columns= {'id','updated_at', 'deleted_at' , 'order_id' , 'manufacture_dept_id', 'order_id', 'prod_not_exist_order_id' })

new_col1 = ['code_order' , 'name_order', 'name_cus' ,'created_at', 'target_quantity' ,  'actual_quantity', 'quantity' ,'complete_percent', 'code_dept', 'name_dept' , 
      'name_not_exist', 'not_exist_create', 'buyer_name', 'style_name' , 'started_at_order' , 'completed_at_order']
   
dfProductionTrackingFix= dfProductionTrackingFix.reindex(columns=new_col1)
   
dfProductionTrackingFix.rename(columns = { 'code_order': 'Mã hàng', 'name_order': 'Tên ĐH', 'name_cus' : 'Tên KH','created_at' : 'Ngày sản xuất' ,'target_quantity' : 'Mục tiêu' , 'actual_quantity' : 'thực tế', 'quantity' : 'Số lượng ĐH',
                                             'complete_percent' : 'Tỉ lệ HT' , 'code_dept' : 'Mã BP' , 'name_dept' : 'Tên BP' , 'name_not_exist': 'ĐH khác' , 'not_exist_create': 'Ngày tạo' , 'started_at_order' : 'Ngày đặt ĐH' ,
                                             'completed_at_order' : 'Ngày giao ĐH'}, inplace = True)
dfProductionTrackingFix

dfProductionTrackingFix = dfProductionTrackingFix.sort_values("Ngày sản xuất" , ascending=False) 
dfProductionTrackingFix.head(2)



,Mã hàng,Tên ĐH,Tên KH,Ngày sản xuất,Mục tiêu,thực tế,Số lượng ĐH,Tỉ lệ HT,Mã BP,Tên BP,ĐH khác,Ngày tạo,buyer_name,style_name,Ngày đặt ĐH,Ngày giao ĐH
1733,DYC-US-230210-E11even-705-7009,DYC-US-230210-E11even-705-7009,DESIGN YOUR CAPS USA LLC,2023-03-03 10:06:03+00:00,1210.0,60.0,202.0,4.96,LEAN 1,May 1,NaN,NaN,NaN,NaN,2023-02-13,2023-03-13
1732,DYC-US-221222-Takuma-680,DYC-US-221222-Takuma-680,DESIGN YOUR CAPS USA LLC,2023-03-03 09:21:02+00:00,83.0,100.0,100.0,120.48,LEAN 1,May 1,NaN,NaN,NaN,NaN,2023-02-15,2023-03-06


In [8]:
dfProductionTrackingFix.head(2)


,Mã hàng,Tên ĐH,Tên KH,Ngày sản xuất,Mục tiêu,thực tế,Số lượng ĐH,Tỉ lệ HT,Mã BP,Tên BP,ĐH khác,Ngày tạo,buyer_name,style_name,Ngày đặt ĐH,Ngày giao ĐH
1733,DYC-US-230210-E11even-705-7009,DYC-US-230210-E11even-705-7009,DESIGN YOUR CAPS USA LLC,2023-03-03 10:06:03+00:00,1210.0,60.0,202.0,4.96,LEAN 1,May 1,NaN,NaN,NaN,NaN,2023-02-13,2023-03-13
1732,DYC-US-221222-Takuma-680,DYC-US-221222-Takuma-680,DESIGN YOUR CAPS USA LLC,2023-03-03 09:21:02+00:00,83.0,100.0,100.0,120.48,LEAN 1,May 1,NaN,NaN,NaN,NaN,2023-02-15,2023-03-06


In [9]:
dfProductionTrackingFix['Ngày sản xuất'] = pd.to_datetime(dfProductionTrackingFix['Ngày sản xuất'])

dfProductionTrackingFix.dtypes

Mã hàng                       object
Tên ĐH                        object
Tên KH                        object
Ngày sản xuất    datetime64[ns, UTC]
Mục tiêu                     float64
thực tế                      float64
Số lượng ĐH                  float64
Tỉ lệ HT                     float64
Mã BP                         object
Tên BP                        object
ĐH khác                       object
Ngày tạo                      object
buyer_name                    object
style_name                    object
Ngày đặt ĐH                   object
Ngày giao ĐH                  object
dtype: object

In [10]:
array = ['LEAN 1', 'LEAN 4', 'LEAN 6', 'LEAN 5', 'LEAN 3', 'LEAN 2', 'LEAN MAU']
dfLean1 = dfProductionTrackingFix[dfProductionTrackingFix['Mã BP'] == 'LEAN 1'].reset_index(drop = True)
dfLean1['Ngày kết thúc'] = np.NaN

dfLean2 = dfProductionTrackingFix[dfProductionTrackingFix['Mã BP'] == 'LEAN 2'].reset_index(drop = True)
dfLean2['Ngày kết thúc'] = np.NaN

dfLean3 = dfProductionTrackingFix[dfProductionTrackingFix['Mã BP'] == 'LEAN 3'].reset_index(drop = True)
dfLean3['Ngày kết thúc'] = np.NaN

dfLean4 = dfProductionTrackingFix[dfProductionTrackingFix['Mã BP'] == 'LEAN 4'].reset_index(drop = True)
dfLean4['Ngày kết thúc'] = np.NaN

dfLean5 = dfProductionTrackingFix[dfProductionTrackingFix['Mã BP'] == 'LEAN 5'].reset_index(drop = True)
dfLean5['Ngày kết thúc'] = np.NaN

dfLean6 = dfProductionTrackingFix[dfProductionTrackingFix['Mã BP'] == 'LEAN 6'].reset_index(drop = True)
dfLean6['Ngày kết thúc'] = np.NaN

dfLeanMau = dfProductionTrackingFix[dfProductionTrackingFix['Mã BP'] == 'LEAN MAU'].reset_index(drop = True)
dfLeanMau['Ngày kết thúc'] = np.NaN


In [11]:
for i in range(1 , len(dfLean1)):
    dfLean1.iloc[i , [16]] = dfLean1['Ngày sản xuất'][i-1]

for i in range(1 , len(dfLean2)):
    dfLean2.iloc[i , [16]] = dfLean2['Ngày sản xuất'][i-1]

for i in range(1 , len(dfLean3)):
    dfLean3.iloc[i , [16]] = dfLean3['Ngày sản xuất'][i-1]

for i in range(1 , len(dfLean4)):
    dfLean4.iloc[i , [16]] = dfLean4['Ngày sản xuất'][i-1]

for i in range(1 , len(dfLean5)):
    dfLean5.iloc[i , [16]] = dfLean5['Ngày sản xuất'][i-1]

for i in range(1 , len(dfLean6)):
    dfLean6.iloc[i , [16]] = dfLean6['Ngày sản xuất'][i-1]

for i in range(1 , len(dfLeanMau)):
    dfLeanMau.iloc[i , [16]] = dfLeanMau['Ngày sản xuất'][i-1]

dfProductionTrackingConcat = pd.concat([dfLean1 , dfLean2 , dfLean3 , dfLean4 , dfLean5, dfLean6, dfLeanMau])

dfProductionTrackingConcat['Ngày sản xuất'] = dfProductionTrackingConcat['Ngày sản xuất'].astype(str)
dfProductionTrackingConcat['Ngày kết thúc'] = dfProductionTrackingConcat['Ngày kết thúc'].astype(str)

dfProductionTrackingConcat
dfProductionTrackingConcat.to_excel('../dataDA/get_production_tracking.xlsx')


In [12]:
dfProductionTrackingConcat.head(2)

,Mã hàng,Tên ĐH,Tên KH,Ngày sản xuất,Mục tiêu,thực tế,Số lượng ĐH,Tỉ lệ HT,Mã BP,Tên BP,ĐH khác,Ngày tạo,buyer_name,style_name,Ngày đặt ĐH,Ngày giao ĐH,Ngày kết thúc
0,DYC-US-230210-E11even-705-7009,DYC-US-230210-E11even-705-7009,DESIGN YOUR CAPS USA LLC,2023-03-03 10:06:03+00:00,1210.0,60.0,202.0,4.96,LEAN 1,May 1,NaN,NaN,NaN,NaN,2023-02-13,2023-03-13,nan
1,DYC-US-221222-Takuma-680,DYC-US-221222-Takuma-680,DESIGN YOUR CAPS USA LLC,2023-03-03 09:21:02+00:00,83.0,100.0,100.0,120.48,LEAN 1,May 1,NaN,NaN,NaN,NaN,2023-02-15,2023-03-06,2023-03-03 10:06:03+00:00


In [13]:
dfWorkingTime = pd.read_csv('../data_csv/WorkingTime.csv')
dfProductionTrackingDetail = pd.read_csv('../data_csv/ProductionTrackingDetail.csv')
dfProductionTrackingDetail = dfProductionTrackingDetail.drop(columns= {'deleted_at' , 'updated_at'}).rename(columns = {'created_at' : 'created_detail' , 'quantity' : 'quantity_detail' ,
                                                                                                       'production_tracking_id' : 'id' ,'manufacture_dept_id' : 'manufacture_dept_id_detail' , 'target_quantity' : "target_quantity_detail" })

dfProductionTrackingDetail
dfWorkingTime = dfWorkingTime.rename(columns= {'id' : 'working_time_id' , 'code' : 'time_working'})
dfWorkingTime


,working_time_id,time_working
0,1,07:00-07:30
1,2,07:30-08:30
2,3,08:30-09:30
3,4,09:30-10:30
4,5,10:30-11:30
5,6,11:30-13:15
6,7,13:15-14:15
7,8,14:15-15:15
8,9,15:15-16:15
9,10,16:15-17:45


In [14]:
dfProdTrackingDetail = dfProductionTracking.merge(dfProductionTrackingDetail , on = ['id' , 'prod_not_exist_order_id' , 'order_id'] , how = 'right')
dfProdTrackingDetail = dfProdTrackingDetail.merge(dfWorkingTime , on = 'working_time_id' , how = 'left')
dfProdTrackingDetail

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,...,name_not_exist,not_exist_create,buyer_name,style_name,quantity_detail,created_detail,working_time_id,manufacture_dept_id_detail,target_quantity_detail,time_working
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.30,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,1,7,0.0,07:00-07:30
1,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.30,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,2,7,0.0,07:30-08:30
2,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.30,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,3,7,0.0,08:30-09:30
3,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.30,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,4,7,0.0,09:30-10:30
4,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.30,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,5,7,0.0,10:30-11:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27739,1760,2023-03-03 10:06:03+00:00,2023-03-03 11:00:07+00:00,NaN,2986.0,NaN,5,1210.0,60.0,4.96,...,NaN,NaN,NaN,NaN,0.0,2023-03-03 10:06:02+00:00,12,5,0.0,18:45-19:45
27740,1760,2023-03-03 10:06:03+00:00,2023-03-03 11:00:07+00:00,NaN,2986.0,NaN,5,1210.0,60.0,4.96,...,NaN,NaN,NaN,NaN,0.0,2023-03-03 10:06:02+00:00,13,5,0.0,19:45-20:45
27741,1760,2023-03-03 10:06:03+00:00,2023-03-03 11:00:07+00:00,NaN,2986.0,NaN,5,1210.0,60.0,4.96,...,NaN,NaN,NaN,NaN,0.0,2023-03-03 10:06:02+00:00,14,5,0.0,20:45-21:45
27742,1760,2023-03-03 10:06:03+00:00,2023-03-03 11:00:07+00:00,NaN,2986.0,NaN,5,1210.0,60.0,4.96,...,NaN,NaN,NaN,NaN,0.0,2023-03-03 10:06:02+00:00,15,5,0.0,21:45-22:00


In [15]:
dfProdTrackingDetail[(dfProdTrackingDetail['name_not_exist'] == '1666975234DH1079') & (dfProdTrackingDetail['id'] == 1)]

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,...,name_not_exist,not_exist_create,buyer_name,style_name,quantity_detail,created_detail,working_time_id,manufacture_dept_id_detail,target_quantity_detail,time_working
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,1,7,0.0,07:00-07:30
1,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,2,7,0.0,07:30-08:30
2,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,3,7,0.0,08:30-09:30
3,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,4,7,0.0,09:30-10:30
4,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,5,7,0.0,10:30-11:30
5,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,19.0,2022-10-22 03:00:00+00:00,6,7,19.0,11:30-13:15
6,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,73.0,2022-10-22 03:00:00+00:00,7,7,110.0,13:15-14:15
7,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,11.0,2022-10-22 03:00:00+00:00,8,7,110.0,14:15-15:15
8,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,215.0,2022-10-22 03:00:00+00:00,9,7,110.0,15:15-16:15
9,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,10,7,110.0,16:15-17:45


In [16]:
dfProdTrackingDetail.head(2)

,id,created_at,updated_at,deleted_at,order_id,prod_not_exist_order_id,manufacture_dept_id,target_quantity,actual_quantity,complete_percent,...,name_not_exist,not_exist_create,buyer_name,style_name,quantity_detail,created_detail,working_time_id,manufacture_dept_id_detail,target_quantity_detail,time_working
0,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,1,7,0.0,07:00-07:30
1,1,2022-10-22 03:00:00+00:00,2022-12-15 10:01:27+00:00,NaN,NaN,1.0,7,927.0,318.0,34.3,...,1666975234DH1079,2022-10-22 03:00:00+00:00,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,2,7,0.0,07:30-08:30


In [17]:
dfProdTrackingDetailFix = dfProdTrackingDetail.drop(columns= {'updated_at','id', 'deleted_at' , 'order_id' , 'prod_not_exist_order_id', 'manufacture_dept_id', 'working_time_id' ,
                                                              'manufacture_dept_id_detail' , 'not_exist_create'})

dfProdTrackingDetailFix

,created_at,target_quantity,actual_quantity,complete_percent,code_order,name_order,name_cus,quantity,started_at_order,completed_at_order,code_dept,name_dept,name_not_exist,buyer_name,style_name,quantity_detail,created_detail,target_quantity_detail,time_working
0,2022-10-22 03:00:00+00:00,927.0,318.0,34.30,NaN,NaN,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,07:00-07:30
1,2022-10-22 03:00:00+00:00,927.0,318.0,34.30,NaN,NaN,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,07:30-08:30
2,2022-10-22 03:00:00+00:00,927.0,318.0,34.30,NaN,NaN,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,08:30-09:30
3,2022-10-22 03:00:00+00:00,927.0,318.0,34.30,NaN,NaN,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,09:30-10:30
4,2022-10-22 03:00:00+00:00,927.0,318.0,34.30,NaN,NaN,NaN,NaN,NaN,NaN,LEAN 3,May 3,1666975234DH1079,PO 38056 GODDESS,PO38056 Shade Goddess_DK_Violet,0.0,2022-10-22 03:00:00+00:00,0.0,10:30-11:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27739,2023-03-03 10:06:03+00:00,1210.0,60.0,4.96,DYC-US-230210-E11even-705-7009,DYC-US-230210-E11even-705-7009,DESIGN YOUR CAPS USA LLC,202.0,2023-02-13,2023-03-13,LEAN 1,May 1,NaN,NaN,NaN,0.0,2023-03-03 10:06:02+00:00,0.0,18:45-19:45
27740,2023-03-03 10:06:03+00:00,1210.0,60.0,4.96,DYC-US-230210-E11even-705-7009,DYC-US-230210-E11even-705-7009,DESIGN YOUR CAPS USA LLC,202.0,2023-02-13,2023-03-13,LEAN 1,May 1,NaN,NaN,NaN,0.0,2023-03-03 10:06:02+00:00,0.0,19:45-20:45
27741,2023-03-03 10:06:03+00:00,1210.0,60.0,4.96,DYC-US-230210-E11even-705-7009,DYC-US-230210-E11even-705-7009,DESIGN YOUR CAPS USA LLC,202.0,2023-02-13,2023-03-13,LEAN 1,May 1,NaN,NaN,NaN,0.0,2023-03-03 10:06:02+00:00,0.0,20:45-21:45
27742,2023-03-03 10:06:03+00:00,1210.0,60.0,4.96,DYC-US-230210-E11even-705-7009,DYC-US-230210-E11even-705-7009,DESIGN YOUR CAPS USA LLC,202.0,2023-02-13,2023-03-13,LEAN 1,May 1,NaN,NaN,NaN,0.0,2023-03-03 10:06:02+00:00,0.0,21:45-22:00


In [18]:
dfProdTrackingDetailFix = dfProdTrackingDetail.drop(columns= {'updated_at','id', 'deleted_at' , 'order_id' , 'prod_not_exist_order_id', 'manufacture_dept_id', 'working_time_id' ,
                                                              'manufacture_dept_id_detail' , 'not_exist_create' , 'style_name'})

new_col1 = ['code_dept' , 'name_dept' , 'code_order' , 'name_order' , 'name_cus' ,'created_at', 'time_working' ,'target_quantity_detail' , 'quantity_detail' , 'target_quantity', 'actual_quantity', 'complete_percent' ,  'quantity' ,
      'name_not_exist', 'not_exist_create', 'buyer_name', 'style_name' ]
   
dfProdTrackingDetailFix= dfProdTrackingDetailFix.reindex(columns=new_col1)
   
dfProdTrackingDetailFix.rename(columns = { 'code_dept': 'Mã chuyền', 'name_dept': 'Tên chuyền', 'code_order' : 'Mã ĐH' ,'name_order' : 'Tên ĐH', 'name_not_exist' : 'Tên ĐH không tồn tại' , 'name_cus' : 'Tên Khách hàng', 'buyer_name' : 'Tên Khách hàng khác',
                                             'created_at' : 'Ngày SX' , 'not_exist_create' : 'Ngày SX đơn hàng khác' , 'time_working' : 'Thời gian SX' , 'target_quantity_detail' : 'SL mục tiêu theo mốc TG' , 'quantity_detail': 'SL theo mốc TG' , 'target_quantity': 'SL mục tiêu tổng' , 
                                             'actual_quantity' : 'SL thực tế' , 'complete_percent' : 'Tỉ lệ hoàn thành' , 'quantity' : 'Số lượng ĐH'}, inplace = True)
dfProdTrackingDetailFix

,Mã chuyền,Tên chuyền,Mã ĐH,Tên ĐH,Tên Khách hàng,Ngày SX,Thời gian SX,SL mục tiêu theo mốc TG,SL theo mốc TG,SL mục tiêu tổng,SL thực tế,Tỉ lệ hoàn thành,Số lượng ĐH,Tên ĐH không tồn tại,Ngày SX đơn hàng khác,Tên Khách hàng khác,style_name
0,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,07:00-07:30,0.0,0.0,927.0,318.0,34.30,NaN,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
1,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,07:30-08:30,0.0,0.0,927.0,318.0,34.30,NaN,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
2,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,08:30-09:30,0.0,0.0,927.0,318.0,34.30,NaN,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
3,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,09:30-10:30,0.0,0.0,927.0,318.0,34.30,NaN,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
4,LEAN 3,May 3,NaN,NaN,NaN,2022-10-22 03:00:00+00:00,10:30-11:30,0.0,0.0,927.0,318.0,34.30,NaN,1666975234DH1079,NaN,PO 38056 GODDESS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27739,LEAN 1,May 1,DYC-US-230210-E11even-705-7009,DYC-US-230210-E11even-705-7009,DESIGN YOUR CAPS USA LLC,2023-03-03 10:06:03+00:00,18:45-19:45,0.0,0.0,1210.0,60.0,4.96,202.0,NaN,NaN,NaN,NaN
27740,LEAN 1,May 1,DYC-US-230210-E11even-705-7009,DYC-US-230210-E11even-705-7009,DESIGN YOUR CAPS USA LLC,2023-03-03 10:06:03+00:00,19:45-20:45,0.0,0.0,1210.0,60.0,4.96,202.0,NaN,NaN,NaN,NaN
27741,LEAN 1,May 1,DYC-US-230210-E11even-705-7009,DYC-US-230210-E11even-705-7009,DESIGN YOUR CAPS USA LLC,2023-03-03 10:06:03+00:00,20:45-21:45,0.0,0.0,1210.0,60.0,4.96,202.0,NaN,NaN,NaN,NaN
27742,LEAN 1,May 1,DYC-US-230210-E11even-705-7009,DYC-US-230210-E11even-705-7009,DESIGN YOUR CAPS USA LLC,2023-03-03 10:06:03+00:00,21:45-22:00,0.0,0.0,1210.0,60.0,4.96,202.0,NaN,NaN,NaN,NaN


In [19]:
dfProdTrackingDetailFix = dfProdTrackingDetailFix.sort_values("Ngày SX" , ascending=False) 
dfProductionTrackingFix = dfProductionTrackingFix.sort_values("Ngày sản xuất" , ascending=False) 
dfProductionTrackingFix.dtypes

Mã hàng                       object
Tên ĐH                        object
Tên KH                        object
Ngày sản xuất    datetime64[ns, UTC]
Mục tiêu                     float64
thực tế                      float64
Số lượng ĐH                  float64
Tỉ lệ HT                     float64
Mã BP                         object
Tên BP                        object
ĐH khác                       object
Ngày tạo                      object
buyer_name                    object
style_name                    object
Ngày đặt ĐH                   object
Ngày giao ĐH                  object
dtype: object

In [20]:
with pd.ExcelWriter('../dataDA/get_production_tracking.xlsx') as writer:  
    dfProdTrackingDetailFix.to_excel(writer, sheet_name='tracking_detail')
    dfProductionTrackingConcat.to_excel(writer, sheet_name='tracking')
